In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

C:\ProgramData\Miniconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X_train = pd.read_csv('dengue_features_train.csv', index_col=None)
X_test = pd.read_csv('dengue_features_test.csv', index_col=None)

In [54]:
X_train['week_start_date'] = pd.to_datetime(X_train['week_start_date'])
X_test['week_start_date'] = pd.to_datetime(X_test['week_start_date'])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [4]:
X_train['month'] = X_train['week_start_date'].dt.month
X_train['quarter'] = X_train['week_start_date'].dt.quarter
X_train = X_train.drop(['week_start_date'], axis=1)
X_test['month'] = X_test['week_start_date'].dt.month
X_test['quarter'] = X_test['week_start_date'].dt.quarter
X_test = X_test.drop(['week_start_date'], axis=1)

In [5]:
city_train = pd.get_dummies(X_train['city'])
X_train = pd.concat([X_train, city_train], axis=1)
X_train = X_train.drop(['city'], axis=1)

In [49]:
city_test = pd.get_dummies(X_test['city'])
X_test = pd.concat([X_test, city_test], axis=1)
X_test = X_test.drop(['city'], axis=1)

In [6]:
X_train.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,month,quarter,iq,sj
0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4,2,0,1
1,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,2,0,1
2,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,5,2,0,1
3,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,5,2,0,1
4,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,5,2,0,1


In [51]:
X_train.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [52]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 25 columns):
year                                     416 non-null int64
weekofyear                               416 non-null int64
week_start_date                          416 non-null object
ndvi_ne                                  373 non-null float64
ndvi_nw                                  405 non-null float64
ndvi_se                                  415 non-null float64
ndvi_sw                                  415 non-null float64
precipitation_amt_mm                     414 non-null float64
reanalysis_air_temp_k                    414 non-null float64
reanalysis_avg_temp_k                    414 non-null float64
reanalysis_dew_point_temp_k              414 non-null float64
reanalysis_max_air_temp_k                414 non-null float64
reanalysis_min_air_temp_k                414 non-null float64
reanalysis_precip_amt_kg_per_m2          414 non-null float64
reanalysis_relative_humidity_per

In [8]:
y_train = pd.read_csv('dengue_labels_train.csv', index_col=None)
y_train = y_train['total_cases']

In [14]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(26, input_dim=26, kernel_initializer='normal', activation='relu'))
	model.add(Dense(13, kernel_initializer='normal', activation='relu'))
	model.add(Dense(5, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [25]:
def basic_model():
	model = Sequential()
	model.add(Dense(26, input_dim=26, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [15]:
seed = 7
np.random.seed(seed)

In [29]:
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

In [44]:
nn_model = KerasRegressor(build_fn=larger_model, nb_epoch=100, batch_size=5, verbose=0)

In [36]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(nn_model, X_train, y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/1
146/146 [==============================] - 0s 935us/step
Epoch 1/1
146/146 [==============================] - 0s 541us/step
Epoch 1/1
146/146 [==============================] - 0s 596us/step
Epoch 1/1
146/146 [==============================] - 0s 655us/step
Epoch 1/1
146/146 [==============================] - 0s 709us/step
Epoch 1/1
146/146 [==============================] - 0s 757us/step
Epoch 1/1
145/145 [==============================] - 0s 859us/step
Epoch 1/1
145/145 [==============================] - 0s 932us/step
Epoch 1/1
145/145 [==============================] - 0s 987us/step
Epoch 1/1
145/145 [==============================] - 0s 997us/step
Results: -2479.06 (3768.86) MSE


In [31]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

ValueError: Expected 2D array, got 1D array instead:
array=[4. 5. 4. ... 1. 1. 4.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
y_train = sc.fit_transform(y_train)

In [53]:
X_test = sc.fit_transform(X_test)

ValueError: could not convert string to float: '2013-06-25'

In [45]:
nn_model.fit(X_train,y_train)

In [50]:
result = nn_model.predict(X_test)

ValueError: Error when checking : expected dense_41_input to have shape (26,) but got array with shape (25,)

In [39]:
result

array([19.282606, 18.72331 , 18.49626 , 18.501005, 18.670094, 19.2663  ,
       19.13949 , 18.844933, 18.838566, 18.823294, 18.92696 , 18.702356,
       18.794388, 18.845942, 19.49294 , 18.806442, 19.62377 , 19.174683,
       19.653992, 20.251656, 19.387669, 21.790445, 19.548704, 19.038132,
       19.695494, 19.647083, 18.908806, 19.499327, 19.103365, 19.529753,
       19.226978, 18.822947, 18.899055, 19.165165, 18.950037, 18.879404,
       18.134428, 18.232363, 18.525167, 18.400919, 18.301752, 18.586214,
       18.58039 , 18.287983, 18.723944, 18.7682  , 18.334162, 18.566154,
       18.337843, 18.337719, 18.336678, 18.563953, 18.645721, 18.578339,
       18.947275, 19.268566, 19.360237, 18.747635, 19.10192 , 19.145193,
       18.999397, 18.772947, 19.09318 , 18.834843, 18.99777 , 19.19121 ,
       18.93082 , 19.068764, 19.95211 , 19.15624 , 19.01581 , 19.425617,
       19.109268, 19.419456, 19.251469, 19.537434, 19.388527, 19.06298 ,
       19.051601, 19.061522, 19.731144, 19.847485, 

In [40]:
X_test = pd.read_csv('dengue_features_test.csv', index_col=None)
result_columns = X_test[['city','year','weekofyear']]
result_cases = pd.DataFrame(result)
result_cases.columns = ['total_cases']
result_df = pd.concat([result_columns, result_cases], axis=1)
result_df['total_cases'] = result_df['total_cases'].round().astype(int)

In [41]:
result_df['total_cases'] = np.where(result_df['total_cases']<0, 0, result_df['total_cases'])

In [42]:
result_df.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,19
1,sj,2008,19,19
2,sj,2008,20,18
3,sj,2008,21,19
4,sj,2008,22,19


In [43]:
result_df.to_csv('keras_basic.csv', index=False)